In [1]:
from __future__ import unicode_literals, print_function
import pandas as pd
import os.path
from nltk.tokenize import word_tokenize
import nltk
import json
import spacy
import plac
spacy.prefer_gpu()
import en_core_web_lg
from spacy_conll import ConllFormatter
nlp = en_core_web_lg.load()
conllformatter = ConllFormatter(nlp)
nlp.add_pipe(conllformatter, last=True)

TRIALPATH = "data/trial"
TRAINPATH = "data/train"

class exerpt:
    def __init__(self, name, txt, ann, tsv, grobid):
        self.name = name
        self.txt = txt
        self.ann = ann
        self.tsv = tsv
        self.grobid = grobid
        self.context = False
        self.posTag = False
        self.doc = nlp(self.txt)
        
    def getContext(self, count = 3):
        if self.context == False: 
            beforeContext = []
            afterContext = []
            self.context = True
            for start,end in zip(self.tsv["startOffset"].values, self.tsv["endOffset"].values ):#self.tsv.iterrows():
                a = getIndexSeperated(exerpt.txt[0],end,count = count)
                b = getIndexSeperated(exerpt.txt[0],start,count = count,forward = False)
                beforeContext.append(self.txt[0][b:start])
                afterContext.append(self.txt[0][end:a])
            self.tsv.insert(3,"beforeContext", beforeContext)
            self.tsv.insert(3,"afterContext", afterContext)
            
    def getPosTag(self):
        if self.context and self.posTag == False: 
            bTag = []
            aTag = []
            self.posTag = True
            for before, after in zip(self.tsv["beforeContext"].values,self.tsv["afterContext"].values):
                bTag.append(nltk.pos_tag(word_tokenize(before)))
                aTag.append(nltk.pos_tag(word_tokenize(after)))
            self.tsv.insert(3,"beforeTag", bTag)
            self.tsv.insert(3,"afterTag", aTag)
            


def readTXTByLine(filepath):
    ftemp = open(filepath, "r", encoding = "utf-8")
    raw = str(ftemp.read())
    ftemp.close()
    return raw

data = {}
for fn in os.listdir(os.path.join(TRIALPATH,"txt")):
        if fn.endswith('.txt'):
            data[fn[:-4]] = exerpt(
                fn[:-4],
                readTXTByLine(os.path.join(TRIALPATH, "txt", fn[:-4] + ".txt")),
                readTXTByLine(os.path.join(TRIALPATH, "ann", fn[:-4] + ".ann")),
                pd.read_csv(os.path.join(TRIALPATH, "tsv", fn[:-4] + ".tsv"), "\t", header = 0 ),
                json.load(open(os.path.join(TRIALPATH, "grobid", fn[:-4] + ".grobid")))
            )
            
for fn in [x for x in os.listdir(os.path.join(TRAINPATH,"text")) if x[:-4]+".tsv" in os.listdir(os.path.join(TRAINPATH,"tsv"))]:
        if fn.endswith('.txt'):
            data[fn[:-4]] = exerpt(
                fn[:-4],
                readTXTByLine(os.path.join(TRAINPATH, "text", fn[:-4] + ".txt")),
                "none",
                pd.read_csv(os.path.join(TRAINPATH, "tsv", fn[:-4] + ".tsv"), "\t", header = 0 ),
                json.load(open(os.path.join(TRAINPATH, "grobid", fn[:-4] + ".grobid")))
            )

In [2]:
for index, row in data['S0012821X12004384-1302'].tsv.iterrows():
    tempAnnot = {}
    tempAnnot["indices"] = [row["startOffset"],row["endOffset"]] 
    tempAnnot["annotSet"] = row["annotSet"]
    tempAnnot["annotId"] = row["annotId"]
    tempAnnot["text"] = row["text"]
    tempAnnot["other"] = row["other"]
    try:
        twitjson["entities"][row["annotType"]].append(tempAnnot)
    except KeyError:
        twitjson["entities"][row["annotType"]] = [tempAnnot] 

NameError: name 'twitjson' is not defined

In [ ]:
# import json 
# for doc in data.values():

#     testjson  = doc.doc.to_json()


#     twitjson = {"full_text": testjson["text"],"entities":{}}


#     for tok in testjson["tokens"]:
#         tempToken = {}
#         tempToken["indices"] = [tok["start"],tok["end"]] 
#         tempToken["category"] = tok["tag"]
#         tempToken["dep"] = tok["dep"]
#         tempToken["id"] = tok["id"]
#         tempToken["head"] = tok["head"]
#         try:
#             twitjson["entities"]["Token"].append(tempToken)
#         except KeyError:
#             twitjson["entities"]["Token"] = [tempToken] 

#     for ent in testjson["ents"]:
#         tempEnt = {}
#         tempEnt["indices"] = [ent["start"],ent["end"]] 
#         try:
#             twitjson["entities"][ent["label"]].append(tempEnt)
#         except KeyError:
#             twitjson["entities"][ent["label"]] = [tempEnt] 

#     for sent in testjson["sents"]:
#         tempSent = {}
#         tempSent["indices"] = [sent["start"],sent["end"]] 
#         try:
#             twitjson["entities"]["sentence"].append(tempSent)
#         except KeyError:
#             twitjson["entities"]["sentence"] = [tempSent] 
            
#     for index, row in doc.tsv.iterrows():
#         tempAnnot = {}
#         tempAnnot["indices"] = [row["startOffset"],row["endOffset"]] 
#         tempAnnot["annotSet"] = row["annotSet"]
#         tempAnnot["annotId"] = row["annotId"]
#         tempAnnot["text"] = row["text"]
#         tempAnnot["other"] = row["other"]
#         try:
#             twitjson["entities"]["MEval-"+row["annotType"]].append(tempAnnot)
#         except KeyError:
#             twitjson["entities"]["MEval-"+row["annotType"]] = [tempAnnot] 

#     json.dump(twitjson, open(f'jsondocs/{doc.name}.json',"w"))

In [ ]:
list(data.keys())

In [3]:
for exerpt in list(data.values()):
    print("Name:", exerpt.name)
    print("grobid")
    for x in exerpt.grobid[1]["measurements"]:
        if x["type"] == "value":
            print(x["quantity"]["offsetStart"],"-",x["quantity"]["offsetEnd"],":",x["quantity"]["rawValue"])
        elif x["type"] == "listc":
            for q in x["quantities"]:
                print(q["offsetStart"],"-",q["offsetEnd"],":",q["rawValue"])
        elif x["type"] == "interval":
            try:
                print(x["quantityLeast"]["offsetStart"],"-",x["quantityLeast"]["offsetEnd"],":",x["quantityLeast"]["rawValue"])
            except KeyError:
                continue
            try:
                print(x["quantityMost"]["offsetStart"],"-",x["quantityMost"]["offsetEnd"],":",x["quantityMost"]["rawValue"])
            except KeyError:
                continue
        else: 
            print("not implemented for type:",x["type"])
    print("gold")            
    for start, end, text, tpe in  zip(exerpt.tsv["startOffset"].values, exerpt.tsv["endOffset"].values,exerpt.tsv["text"].values,exerpt.tsv["annotType"].values):
        if tpe == "Quantity":
            print(start,"-", end, ":", text)
            
            
            
            

Name: S0012821X12004384-1302
grobid
492 - 497 : three
553 - 557 : Five
588 - 595 : 2619.60
597 - 604 : 2617.35
606 - 613 : 2617.44
615 - 622 : 2614.73
628 - 635 : 2614.71
769 - 776 : 2619.60
778 - 785 : 2614.73
791 - 798 : 2614.71
1481 - 1484 : two
1493 - 1500 : 2619.60
1505 - 1512 : 2614.71
1992 - 1995 : two
2115 - 2118 : two
gold
553 - 557 : Five
588 - 637 : 2619.60, 2617.35, 2617.44, 2614.73, and 2614.71 m
769 - 800 : 2619.60, 2614.73, and 2614.71 m
1481 - 1484 : two
1493 - 1512 : 2619.60 and 2614.71
Name: S0012821X12004384-1405
grobid
626 - 628 : 20
775 - 776 : 0
781 - 783 : 80
gold
626 - 628 : 20
775 - 784 : 0% to 80%
Name: S0012821X12004384-1415
grobid
44 - 48 : 2632
52 - 56 : 2618
291 - 292 : 5
gold
39 - 58 : from 2632 to 2618 m
280 - 293 : on average 5%
Name: S0012821X12004384-1594
grobid
162 - 166 : 2618
310 - 312 : 40
gold
156 - 168 : above 2618 m
310 - 314 : 40°N
Name: S0012821X12004384-1599
grobid
81 - 84 : 103
117 - 121 : 2618
gold
76 - 88 : from 103 yrs
111 - 123 : above 

112 - 116 : four
123 - 127 : half
149 - 153 : 1947
158 - 162 : 1999
211 - 212 : 1
218 - 220 : 37
229 - 232 : one
259 - 261 : 27
304 - 307 : one
339 - 340 : 3
343 - 344 : 15
557 - 562 : three
618 - 622 : half
663 - 664 : 2
gold
112 - 127 : four and a half
149 - 162 : 1947 and 1999
211 - 221 : 1% and 37%
229 - 232 : one
259 - 266 : 27 days
304 - 307 : one
339 - 345 : 9 ± 6%
493 - 504 : ∼10–11 days
557 - 562 : three
597 - 622 : within a month and a half
649 - 666 : approximately 2 h
Name: S0019103512003995-2760
grobid
324 - 326 : 11
337 - 338 : 5
gold
324 - 333 : 11 km s−1
337 - 340 : 5Rp
Name: S0019103512003995-3548
grobid
235 - 240 : three
651 - 652 : 1
857 - 861 : 8250
1022 - 1026 : 6000
1033 - 1039 : 11,000
gold
651 - 657 : 1 μbar
851 - 863 : about 8250 K
1008 - 1028 : approximately 6000 K
1033 - 1041 : 11,000 K
Name: S0019103512004009-3488
grobid
550 - 554 : 1300
576 - 577 : 1
812 - 813 : 3
gold
181 - 187 : 3 mbar
550 - 566 : 1300 K to 3500 K
576 - 582 : 1 μbar
812 - 818 : 3 mbar
Nam

100 - 104 : 55.8
177 - 180 : 5-8
gold
100 - 107 : 55.8 Ma
177 - 183 : 5–8 °C
382 - 393 : over <20 ka
464 - 482 : between −2 and −7‰
567 - 573 : 170 ka
954 - 959 : <−60‰
Name: S0012821X12004384-1221
grobid
670 - 673 : 200
679 - 683 : 1000
1070 - 1073 : 0.5
gold
662 - 685 : between 200 and >1000 m
1069 - 1076 : >0.5 km
Name: S0012821X12004384-1232
grobid
339 - 341 : 10
531 - 534 : 100
gold
28 - 36 : >0.5 km)
338 - 344 : <10 cm
515 - 536 : in the order of 100 m
Name: S0012821X12004384-1249
grobid
25 - 29 : 8.47
33 - 43 : 1°50’26.59
507 - 511 : 2605
517 - 521 : 2634
698 - 702 : 2609
810 - 811 : 1
703 - 707 : 2613
974 - 976 : 26
1024 - 1026 : 13
gold
19 - 45 : 57°44’8.47”N; 1°50’26.59”E
502 - 523 : from 2605 m to 2634 m
698 - 709 : 2609–2613 m
797 - 824 : ranging from 1 to 25 per mm
974 - 985 : 26 horizons
1010 - 1039 : approximately 13 pairs per mm
Name: S0012821X12004384-1265
grobid
11 - 13 : 71
512 - 517 : 1/100
594 - 595 : 5
689 - 693 : 0.05
1117 - 1120 : 250
1193 - 1195 : 90
1291 - 129

559 - 561 : 20
667 - 671 : 2013
gold
166 - 174 : 20–30 cm
406 - 409 : six
559 - 564 : 20 cm
Name: S0038071712001010-1044
grobid
629 - 630 : 4
1298 - 1301 : six
2609 - 2611 : 84
gold
626 - 634 : ∼0.4 mm2
2609 - 2616 : 84 days
Name: S0038071712001010-918
grobid
220 - 223 : 9.9
238 - 242 : 7.01
403 - 406 : two
456 - 460 : 25.4
499 - 504 : three
514 - 518 : 23.8
672 - 676 : 16.2
693 - 696 : 9.9
794 - 797 : 7.5
875 - 879 : 14.2
gold
57 - 64 : < 0.001
220 - 230 : 9.9%–16.2%
238 - 242 : 7.01
456 - 461 : 25.4%
514 - 519 : 23.8%
672 - 677 : 16.2%
693 - 697 : 9.9%
794 - 798 : 7.5%
875 - 880 : 14.2%
Name: S0038071712001010-944
grobid
224 - 227 : 115
256 - 259 : 169
287 - 292 : 0.001
gold
224 - 228 : 115%
256 - 260 : 169%
287 - 292 : 0.001
Name: S0038071713001971-1388
grobid
178 - 180 : 55
292 - 294 : 63
gold
178 - 181 : 55%
292 - 295 : 63%
Name: S0038071713001971-1427
grobid
100 - 103 : two
gold
100 - 109 : two years
Name: S016412121300188X-4069
grobid
54 - 58 : 9438
88 - 92 : 7538
186 - 188 : 70

KeyError: 'measurements'

In [4]:
# from spacy import displacy
# import os
# from pathlib import Path
# options = {"fine_grained" : True,
#             "collapse_punct": True,
#           "collapse_phrases": False, 
#           "compact" : True,
#           "word_spacing" : 25,
#           "offset_x" : 50,
#           "arrow_stroke" : 1,
#           "arrow_spacing" : 6,
#           "distance" : 90,
#           "arrow_width": 4.5,
#           "font" : "Times"}

# """
# The following code generates dependency parses and saves them to a dependency parses folder
# It is also possible to do this and have spacy output it to a server port... 
# """
# for exerpt in data.values(): 
#     svg = displacy.render(exerpt.doc, jupyter = False, style = "dep", minify = True, options = options)
#     file_name = exerpt.name + ".svg"
#     output_path = Path(os.path.join("./","data/trial/dep", file_name))
#     output_path.open("w", encoding="utf-8").write(svg)

In [5]:

"""A simple example of extracting relations between phrases and entities using
spaCy's named entity recognizer and the dependency parse. Here, we extract
money and currency values (entities labelled as MONEY) and then check the
dependency tree to find the noun phrase they are referring to – for example:
$9.4 million --> Net income.

Compatible with: spaCy v2.0.0+
Last tested with: v2.2.1
"""




TEXTS = [
    "Net income was $9.4 million compared to the prior year of $2.7 million.",
    "Revenue exceeded twelve billion dollars, with a loss of $1b.",
]


def main(model="en_core_web_sm"):
    nlp = spacy.load(model)
    print("Loaded model '%s'" % model)
    print("Processing %d texts" % len(TEXTS))

    for text in TEXTS:
        doc = nlp(text)
        relations = extract_currency_relations(doc)
        for r1, r2 in relations:
            print("{:<10}\t{}\t{}".format(r1.text, r2.ent_type_, r2.text))


def filter_spans(spans):
    # Filter a sequence of spans so they don't contain overlaps
    # For spaCy 2.1.4+: this function is available as spacy.util.filter_spans()
    get_sort_key = lambda span: (span.end - span.start, -span.start)
    sorted_spans = sorted(spans, key=get_sort_key, reverse=True)
    result = []
    seen_tokens = set()
    for span in sorted_spans:
        # Check for end - 1 here because boundaries are inclusive
        if span.start not in seen_tokens and span.end - 1 not in seen_tokens:
            result.append(span)
        seen_tokens.update(range(span.start, span.end))
    result = sorted(result, key=lambda span: span.start)
    return result

skips = {}
skipKind = {}
def extract_currency_relations(doc):
    # Merge entities and noun chunks into one token
    spans = list(doc.ents) + list(doc.noun_chunks)
    spans = filter_spans(spans)
    with doc.retokenize() as retokenizer:
        for span in spans:
            retokenizer.merge(span)

    relations = []
    tps = ["CARDINAL", "MONEY", "ORDINAL", "PERCENT", "DATE", "TIME", "QUANTITY"]
    for identifiedEnt in filter(lambda w: w.ent_type_ in tps, doc):
        if identifiedEnt.dep_ in ("attr", "dobj"):
            subject = [w for w in identifiedEnt.head.lefts if w.dep_ == "nsubj"]
            if subject:
                subject = subject[0]
                relations.append((subject, identifiedEnt))
        elif identifiedEnt.dep_ == "pobj" and identifiedEnt.head.dep_ == "prep":
            relations.append((identifiedEnt.head.head, identifiedEnt))
        else:
            print("skipped: ", identifiedEnt.dep_,  identifiedEnt.pos_, identifiedEnt)
            try:
                skips[identifiedEnt.dep_] += 1
                skipKind[identifiedEnt.dep_].append(identifiedEnt)
            except KeyError: 
                skips[identifiedEnt.dep_] = 1
                skipKind[identifiedEnt.dep_] = [identifiedEnt]
    return relations

nlp = spacy.load("en_core_web_sm")

In [14]:
for ent in data.values():
    print(ent.name)
    relations = extract_currency_relations(ent.doc)
    for r1, r2 in relations:
        print("{:<10}\t{}\t{}\t{}".format(r1.text, r2.ent_type_,r2.pos_, r2.text))
    print()

S0012821X12004384-1302
skipped:  dep NUM 4
skipped:  dep NUM 5
skipped:  dep NOUN 7 and 8)
skipped:  dep X 4
skipped:  conj NUM 2617.35
skipped:  conj NUM 2617.44
skipped:  conj NUM 2614.73
skipped:  conj NOUN 2614.71 m
skipped:  dep X 5
skipped:  dep NUM 3
skipped:  appos NUM 2619.60
skipped:  conj NUM 2614.73
skipped:  dep NUM 3
skipped:  appos NUM 5a
skipped:  nsubj PROPN the CIE, Fig. 8)
skipped:  nummod NUM 1
skipped:  nsubj PROPN (abundant after the CIE, Fig. 8)
skipped:  nummod NUM 1
skipped:  appos NUM 2619.60
skipped:  conj NUM 2614.71
skipped:  appos NUM 2005
skipped:  dep NUM 3
skipped:  ROOT NUM 6–9
Five samples	CARDINAL	NUM	2619.60

S0012821X12004384-1405
skipped:  npadvmod NUM 2005
skipped:  npadvmod NUM 2005
skipped:  dep NUM 7
skipped:  dep NUM 10
ranging   	PERCENT	NOUN	0% to 80%

S0012821X12004384-1415
skipped:  nummod NUM 1
skipped:  appos NUM 7p
skipped:  npadvmod NUM 1994
skipped:  npadvmod NUM 2005
(DA1      	CARDINAL	NUM	2632
from      	QUANTITY	NOUN	2618 m

S001

drawn     	DATE	NUM	1991
follow-up screenings	DATE	NUM	1997

S0006322312001096-1177
skipped:  appos NUM 22
skipped:  npadvmod NUM 35 to 55
skipped:  prep ADP 1985 to 1988
skipped:  npadvmod NOUN approximately every 5 years
skipped:  nummod NUM 3
skipped:  nummod NUM 5
skipped:  nummod NUM 7
skipped:  appos NUM 2003 to 2004
skipped:  nummod NUM 6967
skipped:  nummod NUM 9 (2008 to 2009
skipped:  nummod NUM 3
skipped:  nummod NUM 3
skipped:  nummod NUM 5
skipped:  nummod NUM 5
skipped:  nummod NUM 7
skipped:  nummod NUM 3
skipped:  nummod NUM 5
skipped:  conj NUM 7
skipped:  nummod NUM 7
skipped:  conj NUM 9
established	DATE	NUM	1985
a response	PERCENT	NOUN	73%
incidence 	DATE	NOUN	phases 5

S0006322312001096-1190
skipped:  nummod NUM 3
skipped:  nummod NUM 5
skipped:  conj NUM 7
skipped:  dep NUM 14–17
skipped:  npadvmod NOUN at least 5 hours
skipped:  appos NOUN 1.4%–3.1%
skipped:  appos NUM 24
skipped:  nsubj NOUN 5 minutes
skipped:  nummod NUM 3
skipped:  conj NUM 5
skipped:  nummod 

skipped:  nsubj PROPN 3a
that      	PERCENT	NOUN	almost 70%

S016412121300188X-4392
skipped:  npadvmod NOUN 100%
A precision	CARDINAL	NUM	1.00

S016412121300188X-4436
skipped:  appos NUM 2005
skipped:  nsubj NOUN 10%
Harman et al., 2009; Islam et al., 2010a,b), a threshold	PERCENT	NOUN	10%

S016412121300188X-4545
skipped:  nummod NUM 4
skipped:  nummod NUM 1
skipped:  nsubj NUM 1
skipped:  nsubj NUM 5
skipped:  nummod NUM 2
skipped:  conj NUM 3
skipped:  nummod NUM 4
skipped:  conj NUM 5
range     	PERCENT	NOUN	11.4% to 2.4%

S016412121300188X-4617
skipped:  appos NOUN 52%
skipped:  npadvmod NOUN 11%

S016412121300188X-5038

S016412121300188X-5066
skipped:  conj NUM 2010b
the precision	PERCENT	NOUN	78% to 95%

S0165587612003680-1078
skipped:  appos NUM 33,42
skipped:  npadvmod NUM 43,43
skipped:  appos NUM 45
skipped:  appos NUM 48
skipped:  appos NUM 49,50
skipped:  appos NUM 51,52
skipped:  nummod NUM 2
account   	PERCENT	NOUN	60%
isolated  	CARDINAL	NUM	46,47

S0165587612003680-953


skipped:  npadvmod NOUN 4 weeks
skipped:  appos NOUN Figure S5H
transplantation	CARDINAL	NUM	S5E

S2211124713006475-841
skipped:  npadvmod NOUN 6 weeks

S2213158213000582-1327
skipped:  meta NUM 1
skipped:  appos NOUN (Fig. 1A
skipped:  appos X 1B
skipped:  appos NUM 2001
skipped:  ROOT X 2
skipped:  nsubjpass PROPN The cortical (Fig. 1C
skipped:  appos PROPN (Fig. 1D
skipped:  ROOT X 3
skipped:  meta X 4
skipped:  nummod NUM 3
skipped:  pobj NOUN S1
skipped:  conj PROPN 1F
skipped:  nsubjpass X 5
skipped:  nsubj NUM 1H
skipped:  ROOT PROPN 1C
voxels    	QUANTITY	NOUN	5 mm

S2213158213000582-1340
skipped:  conj NOUN 94.38%
skipped:  conj NOUN 93.17%
skipped:  dep NUM 1.31
skipped:  ROOT NUM 0.20

S2213671113000738-430
(I–M) Teratoma formation	DATE	NOUN	3 months

S2213671113000738-435
skipped:  appos NOUN 200 μm
skipped:  appos NOUN 50 μm
skipped:  conj NOUN 100 μm

S2213671113000738-445
Incubation	TIME	NOUN	48 hr

S2213671113000738-485
skipped:  dep NUM 6
skipped:  npadvmod NOUN 3 mont

In [7]:
buf=""
for x in skipKind.values():
    for y in x: 
        buf=buf+str(y)+", "
buf

'4, 5, 7 and 8), 4, 5, 3, 3, 3, 7, 10, 9, 2001, 1, between 50 and 100, 2007, 5c, 2, 1993, 2, 2000, 2001, 2, 1, 2, 2011, 3, 1, 2010, 10, 6, 3, 1983, 2, 2012, 2, 2005, 0.5, 2009, 2009, 2, 4, 14–17, 2001, 2, 2, 3, 2, 3, 1991, 2004, 13, 2009, 2011, 2009, 2013, 2010, 2008, 1, 1, 13, 2001, 2009, 2, 9, 3, 1993b, 2006, 4, 2003, 5, 0.4–1.0 nm, 4, 29, 2003, 4, 4, 5, 2007, 4P, 1.31, 6, 1, 4, 6 and 8), 10, 2003, 2003, 2617.35, 2617.44, 2614.73, 2614.71 m, 2614.73, 2614.71, up to 1 week, 95%, 2SD, 5, 9.10, 407 ppm, 6 kg, 13 kg, 0 ppm, 1000 ppm and 2000 ppm, several tens, 0.66 s, 10°, 1536, 1260, 3MgF2→2Mg3NF3(2)Mg3N2+MgF2→2Mg2NF, 2003, 2008, 7, 9, 7, 5, 7, 2.25, 2.14, 9, 84%, 81%, 9, 3 = 5–7 days, 86%, 9, 1.24, approximately 80%, 2619.6 m, 6 kg, 13 kg, 9,12, 14, 2, 2, 20, 15, 169%, 3, 5, 2010b, 82.9%, 67.9%, a third, 0.13%, 5, 20, 8.4 mm, 8.5 mm, GP4, 11.5 m, 22%, 2, 11, less than 2.5 μm, 38 °, 43.22%, 17, 23, 14, C9ORF72, 4R, 1F, 94.38%, 93.17%, 100 μm, 1H, 3B, 3H, 3J, 3L, 2619.60, 5a, 2619.60, 20

In [8]:
skips

{'dep': 87,
 'conj': 82,
 'appos': 266,
 'nsubj': 58,
 'nummod': 107,
 'ROOT': 59,
 'npadvmod': 221,
 'punct': 6,
 'prep': 14,
 'amod': 6,
 'pobj': 12,
 'compound': 4,
 'cc': 2,
 'advcl': 1,
 'meta': 6,
 'det': 1,
 'nmod': 5,
 'advmod': 12,
 'nsubjpass': 13,
 'quantmod': 1,
 'oprd': 2,
 'acomp': 1,
 'case': 1}

In [12]:
data["S0019103512003533-3348"].doc.ents

(CH4, Müller-Wodarg et al, 2006, 2° and, 10°)

In [13]:
list(data["S0019103512003533-3348"].doc.noun_chunks)

[The STIM GCM,
 the transport,
 winds,
 molecular and turbulent diffusion,
 key neutral species,
 H,
 H2,
 He,
 CH4,
 H2O,
 the procedures,
 Müller-Wodarg et al,
 The global spherical grid,
 flexible resolution,
 simulations,
 this study,
 we,
 latitude,
 longitude,
 2° and,
 10°,
 a vertical resolution,
 0.4 scale heights,
 Our time integration step,
 5 s,
 we,
 the code,
 500 Saturn rotations,
 steady state]

In [12]:
list(filter(lambda w: w.ent_type_ , data["S0019103512003533-3348"].doc))

[Müller-Wodarg et al., 2006, 10°, 5 s]